# Approach C Demo: Fine-tuned DeBERTa for NLI

## Imports

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Constants

In [17]:
FT_MODEL_PATH = './drive/MyDrive/Colab/NLU_CW/deberta-finetuned-nli'
TEST_PATH = './drive/MyDrive/Colab/NLU_CW/trial.csv'

## Load Fine-tuned Model

In [4]:
ft_model_tokenizer = AutoTokenizer.from_pretrained(f"{FT_MODEL_PATH}")
ft_model = AutoModelForSequenceClassification.from_pretrained(f"{FT_MODEL_PATH}")

In [10]:
ft_model.to(device)

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

## Test Dataset

In [21]:
test_df = pd.read_csv(TEST_PATH)
test_df.head(10)

,premise,hypothesis,label
0,The City Council plan would increase by nearly...,The City Council will raise taxes 50%.,1
1,It was originally planned to travel farther we...,It was originally planned to travel farther we...,0
2,"Rebuilt in modern times, the mosque serves the...",The mosque was torn down and rebuilt 500 years...,0
3,"'J, it's late, I'm tired, and you're talking n...",J rambled incoherently despite my issues with ...,1
4,The church also boasts one of Italy's most del...,The church has the worst fresco cycles.,0
5,To the east of the palace are the Eglise Saint...,There are also interesting things to the west ...,1
6,my problem i guess is time you know i i talk t...,I am thinking of quitting work to take care of...,1
7,"Bihar, Bengal, and Rajputana all went their se...","Bihar, Bengal and kajputana continued to work ...",0
8,A decrease in the number of poor people in Ohi...,Federal legal-aid in a right of all people.,0
9,The 1992-93 famine lacked the central directio...,The 1992-93 famine occurred because of a lack ...,1


## Generate Predictions

In [27]:
predictions = []

for index, row in test_df.iterrows():
  premise = row['premise']
  hypothesis = row['hypothesis']

  input = ft_model_tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
  output = ft_model(input["input_ids"].to(device))

  prediction = torch.softmax(output["logits"][0], -1).tolist()
  if(prediction[0] < prediction[1]):
    predictions.append(1)
  else:
    predictions.append(0)

In [28]:
test_df['prediction'] = predictions

In [29]:
test_df

,premise,hypothesis,label,prediction
0,The City Council plan would increase by nearly...,The City Council will raise taxes 50%.,1,1
1,It was originally planned to travel farther we...,It was originally planned to travel farther we...,0,0
2,"Rebuilt in modern times, the mosque serves the...",The mosque was torn down and rebuilt 500 years...,0,0
3,"'J, it's late, I'm tired, and you're talking n...",J rambled incoherently despite my issues with ...,1,1
4,The church also boasts one of Italy's most del...,The church has the worst fresco cycles.,0,0
5,To the east of the palace are the Eglise Saint...,There are also interesting things to the west ...,1,1
6,my problem i guess is time you know i i talk t...,I am thinking of quitting work to take care of...,1,1
7,"Bihar, Bengal, and Rajputana all went their se...","Bihar, Bengal and kajputana continued to work ...",0,0
8,A decrease in the number of poor people in Ohi...,Federal legal-aid in a right of all people.,0,1
9,The 1992-93 famine lacked the central directio...,The 1992-93 famine occurred because of a lack ...,1,1
